### This notebook is meant for the generaton of code_vector targets from the encoder branch of a trained AutoEncoder

In [2]:
import os
import torch
import numpy as np
import pandas as pd
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
import sys
sys.path.insert(1, '/home/kseuro/Kai/deeplearnphysics/pytorch/particle_generator/')

# My stuff
import ae
import conv_ae
import utils
from dataloader import LArCV_loader

In [27]:
# Set the GPU (GPU 1 is the best option)
device = torch.device(2)

In [4]:
# Set the root path of the AutoEncoder experiments folder
exp_root = "/media/hdd1/kai/particle_generator/experiments/"

In [5]:
# Set the model type folder (either MLP or Conv model)
model_folder = "larcv_ae/" + "conv_ae/"

In [6]:
# Update the exp_root
exp_root += model_folder

## Get the names of all the experiments in the exp_root folder

In [7]:
exp_paths = []
for path in os.listdir(exp_root):
    exp_paths.append(os.path.join(exp_root, path))

print("-"*60)
for i in range(len(exp_paths)):
    print("\n Exp_{}:".format(str(i)), exp_paths[i], '\n')
    print("-"*60)

------------------------------------------------------------

 Exp_0: /media/hdd1/kai/particle_generator/experiments/larcv_ae/conv_ae/conv_ae_64_4-4-4-code-dim 

------------------------------------------------------------

 Exp_1: /media/hdd1/kai/particle_generator/experiments/larcv_ae/conv_ae/conv_ae_64_1000-epochs 

------------------------------------------------------------

 Exp_2: /media/hdd1/kai/particle_generator/experiments/larcv_ae/conv_ae/conv_ae_128_32-8-8-code-dim 

------------------------------------------------------------


In [8]:
# Set the dir of the particular experiment for which to load a model
exp_dir = exp_paths[0] + "/"

In [9]:
# Create the full path to the experiment
exp_path = os.path.join(exp_root, exp_dir)
print("Experiment path set as: \n{}".format(exp_path))

Experiment path set as: 
/media/hdd1/kai/particle_generator/experiments/larcv_ae/conv_ae/conv_ae_64_4-4-4-code-dim/


In [10]:
# Path to model weights
weights_dir = "weights/"

In [11]:
# Load the config csv as a dict
config_csv = exp_path + "config.csv"
config_df = pd.read_csv(config_csv, delimiter = ",")

In [12]:
# Get the model architecture from config df
n_layers = int(config_df[config_df['Unnamed: 0'].str.contains("n_layers")==True]['0'].values.item())
l_dim    = int(config_df[config_df['Unnamed: 0'].str.contains("l_dim")==True]['0'].values.item())
im_size  = int(config_df[config_df['Unnamed: 0'].str.contains("dataset")==True]['0'].values.item())**2
depth    = int(config_df[config_df['Unnamed: 0'].str.contains("depth")==True]['0'].values.item())
im_dim   = int(np.sqrt(im_size))

In [13]:
print("im_dim: {}, l_dim: {}".format(im_dim, l_dim))

im_dim: 64, l_dim: 4


In [14]:
# Set up AE layer sizes
if 'mlp' in exp_path:
    base = [256] 

    # Compute encoder sizes
    sizes = lambda: [ (yield 2**i) for i in range(n_layers) ]
    enc_sizes = base * n_layers
    enc_sizes = [a*b for a,b in zip(enc_sizes, [*sizes()])][::-1]

    # Update kwarg dicts
    # Decoder is the reverse of the encoder
    ae_kwargs = {'enc_sizes' : enc_sizes, 'l_dim' : l_dim, 'im_size' : im_size, 'dec_sizes' : enc_sizes[::-1]}
else:
    # Compute the depth of the feature maps, based on the number of
    # specified layers. If depth is not divisibe by 4, warn
    depth   = [depth] * n_layers
    divisor = lambda: [ (yield 2**i) for i in range(n_layers) ]
    depth   = [a//b for a,b in zip(depth, [*divisor()])][::-1]
        
    # Update kwarg dicts
    # Decoder is the reverse of the encoder
    ae_kwargs = {'enc_depth':[1]+depth+[l_dim], 'dec_depth':[l_dim]+depth[::-1]+[1],'l_dim':l_dim} 

## Load model checkpoint

In [17]:
# Get checkpoint name(s)
checkpoint_path  = exp_path + weights_dir
checkpoint_names = []
for file in os.listdir(checkpoint_path):
    checkpoint_names.append(os.path.join(checkpoint_path, file))

In [18]:
checkpoint_name_labels = []

In [20]:
print("-"*60)
for i in range(len(checkpoint_names)):
    name = checkpoint_names[i].split('/')[-1]
    checkpoint_name_labels.append(name)
    print("\n{}:".format(str(i)), name, '\n')
    print("-"*60)

------------------------------------------------------------

0: best_conv_ae_ep_650.tar 

------------------------------------------------------------

1: best_conv_ae_ep_600.tar 

------------------------------------------------------------

2: best_conv_ae_ep_999.tar 

------------------------------------------------------------

3: best_conv_ae_ep_700.tar 

------------------------------------------------------------

4: best_conv_ae_ep_850.tar 

------------------------------------------------------------

5: best_conv_ae_ep_900.tar 

------------------------------------------------------------

6: best_conv_ae_ep_950.tar 

------------------------------------------------------------

7: best_conv_ae_ep_750.tar 

------------------------------------------------------------

8: best_conv_ae_ep_800.tar 

------------------------------------------------------------


In [21]:
index = 1
current_checkpoint = checkpoint_names[index]
current_checkpoint_label = checkpoint_name_labels[index]

In [22]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(current_checkpoint)

In [24]:
# Load the model checkpoint
# Keys: ['state_dict', 'epoch', 'optimizer']
checkpoint = torch.load(current_checkpoint)

In [28]:
# Set up model on GPU
if 'mlp' in exp_path:
    model = ae.AutoEncoder(**ae_kwargs).to(device)
else:
    model = conv_ae.ConvAutoEncoder(**ae_kwargs).to(device)

In [29]:
# Load the model's state dictionary
# Note: The IncompatibleKeys(missing_keys=[], unexpected_keys=[]) message indicates that
#       there were no problems in loading the state dictionary. Bit confusing...
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [30]:
# Put the model in evaluation mode
model.eval()

ConvAutoEncoder(
  (encoder): ConvEncoder(
    (conv_blocks): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Conv2d(4, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (3): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): LeakyReLU(negative_slope=0.2)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dila

### If evaluating convolutional model, get the size of the code tensor using a random torch tensor

In [38]:
if 'conv' in exp_path:
    x = torch.randn(1, 1, 64, 64).to(device)
    x = model.encoder(x)
    code = (x.shape[1], x.shape[2], x.shape[3])
    print("Code tensor volume is: {}x{}x{}".format(code[0], code[1], code[2]))

Code tensor volume is:4x2x2


### Create targets from the output of the encoder branch

[PyTorch forums discussion on layerwise viz](https://discuss.pytorch.org/t/how-to-visualize-fully-connected-layers-as-images/13626/2)
- In order to generate a variety of targets, we wish to save the output of the encoder's last layer
- Since the output activations will vary depending on the input data image, we will generate a latent vector for each training example in the training dataset

## Set up LArCV1 dataloader

In [48]:
# Path to the training data
test_data = "/media/hdd1/kai/particle_generator/larcv_data/train/larcv_png_{}/".format(im_dim)
print("Training data will be loaded from: \n{}".format(test_data))

Training data will be loaded from: 
/media/hdd1/kai/particle_generator/larcv_data/train/larcv_png_64/


In [49]:
# Set up the torch dataloader
loader_kwargs = {'num_workers' : 2, 'batch_size': 1, 'shuffle': True}
test_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5],[0.5])])
test_dataset    = LArCV_loader(root = test_data, transforms = test_transforms)

Image conversion flag is: L
Images will be loaded from subfolder of: /media/hdd1/kai/particle_generator/larcv_data/train/larcv_png_64/


In [50]:
dataloader = DataLoader(test_dataset, **loader_kwargs)

## Create directory for saving code layer output targets

In [42]:
# Specify the directory where the code vectors should be saved
deploy_dir = "/media/hdd1/kai/particle_generator/code_vectors/"

In [43]:
if 'mlp' in exp_path:
    deploy_dir += "mlp_ae/code_vectors_{}_{}/".format(im_dim, l_dim)
else:
    deploy_dir += "conv_ae/code_vectors_{}_{}_{}_{}/".format(im_dim, code[0], code[1], code[2])

In [ ]:
# Create the save directory, if it doesn't already exist
os.mkdir(deploy_dir)

In [44]:
if 'mlp' in exp_path:
    deploy_dir += "code_vectors_{}_{}/".format(im_dim, l_dim)
else:
    deploy_dir += "code_vectors_{}_{}_{}_{}/".format(im_dim, code[0], code[1], code[2])

In [ ]:
os.mkdir(deploy_dir)

In [45]:
print("Deploy samples will be saved to:\n{}".format(deploy_dir))

Deploy samples will be saved to:
/media/hdd1/kai/particle_generator/code_vectors/conv_ae/code_vectors_64_4_2_2/code_vectors_64_4_2_2/


## Generate deploy targets by looping over the dataloader using only the encoder

In [51]:
# Codes will be a list of numpy arrays of 32-bit floats
codes = []

In [52]:
for idx, image in enumerate(dataloader):

    # Flatten image into a vector, if mlp
    if 'mlp' in exp_path:
        image = image.view(loader_kwargs['batch_size'], -1).to(device)
    else:
        image = image.to(device)
    
    # Get the output of just the encoder
    code = model.encoder(image)
    
    # If using conv model -- flatten tensor
    if 'conv' in exp_path:
        code = code.view(loader_kwargs['batch_size'], -1).to(device)

    # Save the output tensor to a list
    codes.append(code.detach().cpu().numpy())

In [53]:
print("Generated {} code vectors from {} training images".format(len(codes), len(dataloader)))

Generated 53943 code vectors from 53943 training images


In [54]:
codes[0].shape

(1, 16)

## Save the list of code vectors to disk
- We then use the built-in np.save function to store the np array as a .npy file
- The numpy array of floats can be read back losslessly using np.load("float_file.npy")
- This process will require the downstream creation of a [custom dataset](https://pytorch.org/tutorials/beginner/data_loading_tutorial.html)

In [ ]:
for idx, arr in enumerate(codes):
    file_name = deploy_dir + "target_{}.npy".format(idx)
    np.save(file_name, arr)

In [ ]:
# Load a test vector
x = np.load(deploy_dir + "target_0.npy")
print(type(x))
print(x)